In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pybaseball
from pybaseball import statcast
pybaseball.cache.enable()

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
#Marcus added
def get_batter_stats(data, batter_stats = ["Date","G", "PA", "AB", "R", "H", "2B", "3B", "HR", "RBI", "BB", "IBB", "SO", "HBP", "SH", "SF", "GDP", "SB", "CS", "BA", "OBP", "SLG", "OPS","mlbID"]):
    batting_stats = pd.read_csv("batter2023_03_30.csv")
    batting_stats["Date"] = pd.to_datetime(batting_stats["Date"])

    
    bat_clean = batting_stats.loc[:,batter_stats]


    batters = ["away_b1", "away_b2", "away_b3", "away_b4", "away_b5", "away_b6", "away_b7", "away_b8", "away_b9", "home_b1", "home_b2", "home_b3", "home_b4", "home_b5", "home_b6", "home_b7", "home_b8", "home_b9"]
    for batter in batters:
        data = data.merge(bat_clean, how="left", left_on= ["date",batter], right_on=["Date","mlbID"], suffixes=("","_"+batter))

            
    
    data = data.rename(columns = lambda col: col + "_away_b1" if col in batter_stats else col)
    for batter in batters:
        data = data.drop(columns=["Date_"+batter, "mlbID_" + batter])
    return data

In [42]:
#get games and dates
def get_previous_n_games(games, n, filename = "", save_file = False):
    results = pd.read_csv("data_with_batter_stats.csv")
    home_teams, away_teams = results.copy(), results.copy()


    home_teams["team"] = home_teams["home_team"]
    away_teams["team"] = away_teams["away_team"]

    games = pd.concat([home_teams[["game_pk","date","team"]], away_teams[["game_pk","date","team"]]])

    games["date"] = pd.to_datetime(games["date"])

    games.sort_values(by = ["team","date"], inplace = True, axis = 0, ascending = [True, True])
    prev_col = "game_pk"

    for i in range(n):
        column_name = f"prev_{i}_game_pk"
        games[column_name] = games.groupby('team')[prev_col].shift()
        prev_col = column_name
    
    prev_10_games = results[["game_pk","date","away_team","home_team","home_score","away_score","home_result"]]


    if save_file:
        games.to_csv(filename, index = False)
    
    return games

In [43]:
results = pd.read_csv("data_with_batter_stats.csv")
results.head()

,Unnamed: 0,game_pk,home_result,date,away_team,home_team,away_score,home_score,away_starting_pitcher,home_starting_pitcher,...,HBP_home_b9,SH_home_b9,SF_home_b9,GDP_home_b9,SB_home_b9,CS_home_b9,BA_home_b9,OBP_home_b9,SLG_home_b9,OPS_home_b9
0,0,718782,L,2023-03-30,BAL,BOS,10,9,502043,446372,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.00,0.000
1,1,718781,W,2023-03-30,SF,NYY,0,5,657277,543037,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.333,0.00,0.333
2,2,718767,W,2023-03-30,CLE,SEA,0,3,669456,622491,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.667,0.50,1.167
3,3,718768,L,2023-03-30,CWS,HOU,3,2,656302,664285,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000,0.00,0.000
4,4,718770,W,2023-03-30,AZ,LAD,2,8,668678,628711,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.500,0.75,1.250


In [4]:
away_batters = pd.read_csv("batting_data_away.csv")
home_batters = pd.read_csv("batting_data_home.csv")


print(home_batters.head())

   game_pk  batter  ab  bb  hbp  single  double  triple  hr  sf
0   718767  677594   4   0    0       0       0       0   0   0
1   718767  543939   3   0    1       0       0       0   0   0
2   718767  664034   4   0    0       1       1       0   1   0
3   718767  606192   4   0    0       0       0       0   0   0
4   718767  663728   4   0    0       1       0       0   0   0
